In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import cv2
import math

In [ ]:
def nan_check(value):
    for x in value:
        if math.isnan(x) == True:
            return True
    return False
        
            

In [ ]:
def data_extractor(path):
    image = []
    keys = []
    data = np.array(pd.read_csv(path))
    data = np.reshape(data, (len(data),1,31))
    for i in tqdm(range(len(data))):
        if (data[i].shape[1] == 31 and nan_check(data[i][0,:30]) == False):
            points = [int(u) for u in data[i][0,:30]]
            keys.append(points)
            temp_image = data[i][0,30]
            temp_image=temp_image.split(' ')
            j=[]
            for value in temp_image:
                j.append(int(value))
            j= np.array(j).reshape(96,96,1)
            image.append(j)
    return image,keys

In [ ]:
def for_test(path):
    image = []
    data = np.array(pd.read_csv(path))
    for i in tqdm(range(len(data))):
        temp = data[i][1]
        temp = temp.split(' ')
        j=[]
        for value in temp:
            j.append(int(value))
        j= np.array(j).reshape(96,96,1)
        image.append(j)
    return image

In [ ]:
training_path = r"C:\Users\bisha\Desktop\facial-keypoints\training.csv"
print("generating the training data:")
train_images,train_keys = data_extractor(training_path)

In [ ]:
testing_path = r"C:\Users\bisha\Desktop\facial-keypoints\test.csv"
print("generating the testing data:")
test_images = for_test(testing_path)

In [ ]:
training_images = np.array(train_images)

training_keys = np.array(train_keys)

test_images = np.array(test_images)

print("training data:")
print(training_images.shape)
print(training_keys.shape)

print("testing data:")
print(test_images.shape)

In [ ]:
img_inputs = tf.keras.Input(shape=(96, 96, 1))
conv=tf.keras.layers.Conv2D(16,(3,3),padding='same',input_shape=(96,96,1),activation='relu')
x=conv(img_inputs)
x=tf.keras.layers.MaxPooling2D(2)(x)
x=tf.keras.layers.Conv2D(32,(3,3),padding='same',activation='relu')(x)
x=tf.keras.layers.MaxPooling2D(2)(x)
x=tf.keras.layers.Conv2D(64,(3,3),padding='same',activation='relu')(x)
x=tf.keras.layers.MaxPooling2D(2)(x)
x=tf.keras.layers.Conv2D(128,(3,3),padding='same',activation='relu')(x)
x=tf.keras.layers.MaxPooling2D(2)(x)
x=tf.keras.layers.Conv2D(128,(3,3),padding='same',activation='relu')(x)
x=tf.keras.layers.MaxPooling2D(2)(x)
# x=tf.keras.layers.Conv2D(70,(3,3),padding='same',activation='relu')(x)
# x=tf.keras.layers.MaxPooling2D(2,2)(x)
# x=tf.keras.layers.Conv2D(80,(3,3),padding='same',activation='relu')(x)
x=tf.keras.layers.Flatten()(x)
o1=tf.keras.layers.Dense(512,activation='relu')(x)
# o1=tf.keras.layers.Dense(80,activation='relu')(o1)
o1=tf.keras.layers.Dropout(0.2)(o1)
o1=tf.keras.layers.Dense(30)(o1)

model = tf.keras.models.Model(inputs=img_inputs, outputs= o1)

In [ ]:
model.compile(optimizer="adam", loss="mean_squared_error",metrics=["accuracy"])

In [ ]:
history=model.fit(training_images, training_keys, batch_size = 128, epochs=150, validation_split = 0.02 )

In [ ]:
#training visualization

plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.show()

In [ ]:
#saving the model
model.save(r"C:\Users\bisha\Desktop\facial-keypoints\code\saved_model\key_model.h5")

In [ ]:
#image key plotting...
# x is a image matrix
#k is the key values
x=[]
k=[]
color = 255
isClosed = True
thickness = 4
x = test_images[1]
k = model.predict(test_images[1])
#defining face shape:
p1 = [[int(k[4]),int(k[5])] , [int(k[12]),int(k[13])] , [int(k[14]),int(k[15])] , [int(k[6]),int(k[7])], [int(k[0]),int(k[1])]]
p2 = [[int(k[8]),int(k[9])] , [int(k[16]),int(k[17])] , [int(k[18]),int(k[19])] , [int(k[10]),int(k[11])], [int(k[2]),int(k[3])]]
p3 = [[int(k[20]),int(k[21])], [int(k[4]),int(k[5])] , [int(k[0]),int(k[1])], [int(k[6]),int(k[7])]]
p4 = [[int(k[20]),int(k[21])], [int(k[8]),int(k[9])] , [int(k[2]),int(k[3])], [int(k[10]),int(k[11])]]
p5 = [[int(k[10]),int(k[11])] , [int(k[20]),int(k[21])], [int(k[22]),int(k[23])], [int(k[6]),int(k[7])], [int(k[24]),int(k[25])]]
p6 = [[int(k[22]),int(k[23])], [int(k[26]),int(k[27])] , [int(k[24]),int(k[25])], [int(k[28]),int(k[29])]]
feature=[p1,p2,p3,p4,p5,p6]
for i in feature:
    x = cv2.polylines(np.reshape(np.array(x),(96,96,1)).astype('uint8'), [np.array(i)], isClosed, color)
cv2.imshow('x', x) 
cv2.waitKey(0)
cv2.destroyAllWindows()